In [63]:
''''import dask dataframe'''
import dask.dataframe as dd

In [64]:
'''open sm-all_bacilli.tsv using dask'''
df = dd.read_csv('sm-all_bacilli.tsv', sep='\t', header=None, dtype={8: 'object'})

In [65]:
columns = ["Protein_accession", "Sequence_MD5_digest", "Sequence_length", "Analysis", "Signature_accession", "Signature_description", "Start_location", "Stop_location", "Score", "Status", "Date", "InterPro_annotations_accession", "InterPro_annotations_description", "GO_annotations", "Pathways_annotations"]
df.columns = columns
df.head()

,Protein_accession,Sequence_MD5_digest,Sequence_length,Analysis,Signature_accession,Signature_description,Start_location,Stop_location,Score,Status,Date,InterPro_annotations_accession,InterPro_annotations_description,GO_annotations,Pathways_annotations
0,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,TIGRFAM,TIGR03882,cyclo_dehyd_2: bacteriocin biosynthesis cyclod...,2,131,1.6E-21,T,25-04-2022,IPR022291,"Bacteriocin biosynthesis, cyclodehydratase domain",-,-
1,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,TIGRFAM,TIGR03604,TOMM_cyclo_SagD: thiazole/oxazole-forming pept...,161,547,0.0,T,25-04-2022,IPR027624,"Thiazole/oxazole-forming peptide maturase, Sag...",-,-
2,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,ProSiteProfiles,PS51664,YcaO domain profile.,159,547,75.396477,T,25-04-2022,IPR003776,YcaO-like domain,-,-
3,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,Gene3D,G3DSA:3.30.160.660,-,311,452,9.1E-46,T,25-04-2022,-,-,NaN,NaN
4,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,Gene3D,G3DSA:3.30.40.250,-,191,274,9.1E-46,T,25-04-2022,-,-,NaN,NaN


In [66]:
data = df.copy()
data = data.drop(["Sequence_MD5_digest", "Analysis", "Signature_accession", "Signature_description", "Score", "Status", "Date", "InterPro_annotations_description", "GO_annotations", "Pathways_annotations"], axis=1)
data = data.dropna()
'''drop every row containing '-' in the column 'InterPro_annotations_accession'''
data = data[data['InterPro_annotations_accession'] != '-']
data.head()

,Protein_accession,Sequence_length,Start_location,Stop_location,InterPro_annotations_accession
0,gi|29898682|gb|AAP11954.1|,547,2,131,IPR022291
1,gi|29898682|gb|AAP11954.1|,547,161,547,IPR027624
2,gi|29898682|gb|AAP11954.1|,547,159,547,IPR003776
6,gi|29898682|gb|AAP11954.1|,547,161,501,IPR003776
8,gi|29898682|gb|AAP11954.1|,547,161,501,IPR003776


In [67]:
'''create column, which substracts columns "Start_location" and "Stop_location"'''
data['Length %'] = (data['Stop_location'] - data['Start_location']) / data['Sequence_length']

In [68]:
data.head()

,Protein_accession,Sequence_length,Start_location,Stop_location,InterPro_annotations_accession,Length %
0,gi|29898682|gb|AAP11954.1|,547,2,131,IPR022291,0.235832
1,gi|29898682|gb|AAP11954.1|,547,161,547,IPR027624,0.705667
2,gi|29898682|gb|AAP11954.1|,547,159,547,IPR003776,0.709324
6,gi|29898682|gb|AAP11954.1|,547,161,501,IPR003776,0.621572
8,gi|29898682|gb|AAP11954.1|,547,161,501,IPR003776,0.621572


In [69]:
'''if length % is greater than 0.9, greather_than column is 1, else 0'''
data['greater_than'] = data['Length %'] >= 0.9

In [70]:
data.head()

,Protein_accession,Sequence_length,Start_location,Stop_location,InterPro_annotations_accession,Length %,greater_than
0,gi|29898682|gb|AAP11954.1|,547,2,131,IPR022291,0.235832,False
1,gi|29898682|gb|AAP11954.1|,547,161,547,IPR027624,0.705667,False
2,gi|29898682|gb|AAP11954.1|,547,159,547,IPR003776,0.709324,False
6,gi|29898682|gb|AAP11954.1|,547,161,501,IPR003776,0.621572,False
8,gi|29898682|gb|AAP11954.1|,547,161,501,IPR003776,0.621572,False


In [71]:
'''make protein_90% df with column "Length %" > 0.9'''
protein_90 = data[data['Length %'] >= 0.9]
protein_0 = data[data['Length %'] < 0.9]
protein_90.head()

,Protein_accession,Sequence_length,Start_location,Stop_location,InterPro_annotations_accession,Length %,greater_than
10,gi|29894058|gb|AAP07350.1|,233,1,231,IPR039420,0.987124,True
36,gi|29897770|gb|AAP11045.1|,399,18,391,IPR004636,0.934837,True
37,gi|29897770|gb|AAP11045.1|,399,27,391,IPR005814,0.912281,True
38,gi|29897770|gb|AAP11045.1|,399,14,391,IPR015424,0.944862,True
40,gi|29897770|gb|AAP11045.1|,399,17,391,IPR005814,0.937343,True


In [72]:
protein_0.head()

,Protein_accession,Sequence_length,Start_location,Stop_location,InterPro_annotations_accession,Length %,greater_than
0,gi|29898682|gb|AAP11954.1|,547,2,131,IPR022291,0.235832,False
1,gi|29898682|gb|AAP11954.1|,547,161,547,IPR027624,0.705667,False
2,gi|29898682|gb|AAP11954.1|,547,159,547,IPR003776,0.709324,False
6,gi|29898682|gb|AAP11954.1|,547,161,501,IPR003776,0.621572,False
8,gi|29898682|gb|AAP11954.1|,547,161,501,IPR003776,0.621572,False


In [75]:
protein_90 = protein_90.drop(['Sequence_length', 'Start_location', 'Stop_location', 'greater_than', 'Length %'], axis=1)
protein_0 = protein_0.drop(['Sequence_length', 'Start_location', 'Stop_location', 'greater_than', 'Length %'], axis=1)


In [76]:
protein_90.head()

,Protein_accession,InterPro_annotations_accession
10,gi|29894058|gb|AAP07350.1|,IPR039420
36,gi|29897770|gb|AAP11045.1|,IPR004636
37,gi|29897770|gb|AAP11045.1|,IPR005814
38,gi|29897770|gb|AAP11045.1|,IPR015424
40,gi|29897770|gb|AAP11045.1|,IPR005814


In [96]:
merged = protein_90.merge(protein_0, on='Protein_accession', how='outer')
merged.head()

,Protein_accession,InterPro_annotations_accession_x,InterPro_annotations_accession_y
0,gi|29894058|gb|AAP07350.1|,IPR039420,IPR001867
1,gi|29894058|gb|AAP07350.1|,IPR039420,IPR001867
2,gi|29894058|gb|AAP07350.1|,IPR039420,IPR001867
3,gi|29894058|gb|AAP07350.1|,IPR039420,IPR001867
4,gi|29894058|gb|AAP07350.1|,IPR039420,IPR001789


In [97]:
merged = merged.drop_duplicates()

In [99]:
merged.head()

,Protein_accession,InterPro_annotations_accession_x,InterPro_annotations_accession_y
0,gi|29894058|gb|AAP07350.1|,IPR039420,IPR001867
4,gi|29894058|gb|AAP07350.1|,IPR039420,IPR001789
5,gi|29894058|gb|AAP07350.1|,IPR039420,IPR036388
6,gi|29894058|gb|AAP07350.1|,IPR039420,IPR011006
9,gi|29897770|gb|AAP11045.1|,IPR004636,IPR015421


In [111]:
columns = merged.InterPro_annotations_accession_y.values.compute()
rows = merged.InterPro_annotations_accession_x.values.compute()